<a href="https://colab.research.google.com/github/echodpp/DRL_Application/blob/main/final/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import time
import numpy as np
import json
import openai
import random
import argparse

In [ ]:
rseed = 114514
random.seed(rseed)

In [ ]:
def read_json(file):
    with open(file) as f:
        return json.load(f)

def write_json(data, file):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

data_ml_100k = read_json("/content/drive/MyDrive/Final_Project/ml_100k.json")

In [ ]:
print (data_ml_100k[0][0])
print (data_ml_100k[0][1])
print (len(data_ml_100k))

My Best Friend's Wedding | The English Patient | Face/Off | Psycho | The Princess Bride | Kids | Kids in the Hall: Brain Candy | Scream | The Craft | The Rock | The Shining | Mission: Impossible | Reality Bites | Aliens | Indiana Jones and the Last Crusade | One Flew Over the Cuckoo's Nest | In the Name of the Father | Return of the Jedi | The Silence of the Lambs | The Usual Suspects | Dances with Wolves | Monty Python's Life of Brian | Monty Python and the Holy Grail | The Full Monty | Clockwork Orange, A | Apocalypse Now | Trainspotting | The Doors | Mr. Holland's Opus | Independence Day (ID4) | Liar Liar | Titanic | What's Eating Gilbert Grape | The Saint | Donnie Brasco | The Empire Strikes Back | Beautiful Girls | Mars Attacks! | Broken Arrow | Amistad | The Long Kiss Goodnight | French Kiss | The Maltese Falcon | Dazed and Confused | Strange Days
Starship Troopers
943


In [ ]:
open_ai_keys = ['sk-tKzqyWoHQaGh4b9a498267B9Ca7f403799E34fFf7aA330Ea']
openai.api_base = "https://api.aiguoguo199.com/v1"
open_ai_keys_index = 0
openai.api_key = open_ai_keys[open_ai_keys_index]

In [ ]:
u_item_dict = {}
u_item_p = 0
for elem in data_ml_100k:
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        if movie not in u_item_dict:
            u_item_dict[movie] = u_item_p
            u_item_p +=1
print (len(u_item_dict))
u_item_len = len(u_item_dict)

user_list = []
for i, elem in  enumerate(data_ml_100k):
    item_hot_list = [0 for ii in range(u_item_len)]
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        item_pos = u_item_dict[movie]
        item_hot_list[item_pos] = 1
    user_list.append(item_hot_list)
user_matrix = np.array(user_list)
user_matrix_sim = np.dot(user_matrix, user_matrix.transpose())


pop_dict = {}
for elem in data_ml_100k:
    # elem = data_ml_100k[i]
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        if movie not in pop_dict:
              pop_dict[movie] = 0
        pop_dict[movie] += 1



i_item_dict = {}
i_item_id_list = []
i_item_user_dict = {}
i_item_p = 0
for i, elem in  enumerate(data_ml_100k):
    seq_list = elem[0].split(' | ')
    for movie in seq_list:
        if movie not in i_item_user_dict:
            item_hot_list = [0. for ii in range(len(data_ml_100k))]
            i_item_user_dict[movie] = item_hot_list
            i_item_dict[movie] = i_item_p
            i_item_id_list.append(movie)
            i_item_p+=1
#         item_pos = item_dict[movie]
        i_item_user_dict[movie][i] += 1
#     user_list.append(item_hot_list)
i_item_s_list = []
for item in i_item_id_list:
    i_item_s_list.append(i_item_user_dict[item])
#     print (sum(item_user_dict[item]))
item_matrix = np.array(i_item_s_list)
item_matrix_sim = np.dot(item_matrix, item_matrix.transpose())

id_list =list(range(0,len(data_ml_100k)))

1493


In [ ]:
### user filtering
def sort_uf_items(target_seq, us, num_u, num_i):

    candidate_movies_dict = {}
    sorted_us = sorted(list(enumerate(us)), key=lambda x: x[-1], reverse=True)[:num_u]
    dvd = sum([e[-1] for e in sorted_us])
    for us_i, us_v in sorted_us:
        us_w = us_v * 1.0/dvd
#         print (us_i)
        us_elem = data_ml_100k[us_i]
#         print (us_elem[0])
#         assert 1==0
        us_seq_list = us_elem[0].split(' | ')#+[us_elem[1]]

        for us_m in us_seq_list:
#             print (f"{us_m} not in {target_seq}, {us_m not in target_seq}")
#             break

            if us_m not in target_seq:
                if us_m not in candidate_movies_dict:
                    candidate_movies_dict[us_m] = 0.
                candidate_movies_dict[us_m]+=us_w

#         assert 1==0

    candidate_pairs = list(sorted(candidate_movies_dict.items(), key=lambda x:x[-1], reverse=True))
#     print (candidate_pairs)
    candidate_items = [e[0] for e in candidate_pairs][:num_i]
    return candidate_items

In [ ]:
### item filtering
def soft_if_items(target_seq, num_i, total_i, item_matrix_sim, item_dict):
    candidate_movies_dict = {}
    for movie in target_seq:
#         print('ttt:',movie)
        sorted_is = sorted(list(enumerate(item_matrix_sim[item_dict[movie]])), key=lambda x: x[-1], reverse=True)[:num_i]
        for is_i, is_v in sorted_is:
            s_item = i_item_id_list[is_i]

            if s_item not in target_seq:
                if s_item not in candidate_movies_dict:
                    candidate_movies_dict[s_item] = 0.
                candidate_movies_dict[s_item] += is_v
#             print (item_id_list[is_i], candidate_movies_dict)
    candidate_pairs = list(sorted(candidate_movies_dict.items(), key=lambda x:x[-1], reverse=True))
#     print (candidate_pairs)
    candidate_items = [e[0] for e in candidate_pairs][:total_i]
#     print (candidate_items)
    return candidate_items

In [ ]:
'''
In order to economize, our initial step is to identify user sequences that exhibit a high probability of obtaining accurate predictions from GPT-3.5 based on their respective candidates.
Subsequently, we proceed to utilize the GPT-3.5 API to generate predictions for these promising user sequences.
'''
results_data_15 = []
length_limit = 8
num_u= 12
total_i = 19
count = 0
total = 0
cand_ids = []
for i in id_list[:1000]:
    elem = data_ml_100k[i]
    seq_list = elem[0].split(' | ')

    candidate_items = sort_uf_items(seq_list, user_matrix_sim[i], num_u=num_u, num_i=total_i)

#     print (elem[-1], '-',seq_list[-1])

    if elem[-1] in candidate_items:
#         print ('HIT: 1')
        count += 1
        cand_ids.append(i)
    else:
        pass
#         print ('HIT: 0')
    total +=1
print (f'count/total:{count}/{total}={count*1.0/total}')
print ('-----------------\n')

count/total:170/943=0.18027571580063625
-----------------



In [ ]:
# message = [
#     {"role": "system", "content": "You are a helpful movie recommendation expert."},
#     {"role": "user", "content": in_text},
# ]

temp_1 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)?
Answer:
"""

temp_2 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)?
Answer: {}.
Step 2: Selecting the most featured movies from the watched movies according to my preferences (Format: [no. a watched movie.]).
Answer:
"""

temp_3 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)?
Answer: {}.
Step 2: Selecting the most featured movies (at most 5 movies) from the watched movies according to my preferences in descending order (Format: [no. a watched movie.]).
Answer: {}.
Step 3: Can you recommend 10 movies from the Candidate Set similar to the selected movies I've watched (Format: [no. a watched movie - a candidate movie])?.
Answer:
"""

prompt_1 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Step 1: Please help me summarize the style and main actors of the movie I watched (Format: [no. a watched movie - style and main actors]).
Answer:
"""

prompt_2 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Step 1: Please help me summarize the style and main actors of the movie I watched (Format: [no. a watched movie - style and main actors]).
Answer: {}.
Step 2: Please think step by step, and recommend 10 movies from the Candidate Set that have the similar style and same actors to the movies I've watched (Format: [no. a watched movie - a candidate movie])?.
Answer:
"""

In [ ]:
count = 0
total = 0
results_data = []
for i in cand_ids[:]:#[:10] + cand_ids[49:57] + cand_ids[75:81]:
    elem = data_ml_100k[i]
    seq_list = elem[0].split(' | ')[::-1]

    candidate_items = sort_uf_items(seq_list, user_matrix_sim[i], num_u=num_u, num_i=total_i)
    random.shuffle(candidate_items)

    # input_1 = temp_1.format(', '.join(candidate_items), ', '.join(seq_list[-length_limit:]))
    input_1 = prompt_1.format(', '.join(candidate_items), ', '.join(seq_list[-length_limit:]))


    message_1 = [
          {"role": "system", "content": "You are a helpful movie recommendation expert."},
          {"role": "user", "content": input_1},
    ]

    try_nums = 5
    kk_flag = 1
    while try_nums:
        try:
            response = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=input_1,
                      max_tokens=512,
                      temperature=0,
                      top_p=1,
                      frequency_penalty=0,
                      presence_penalty=0,
                      n = 1,
                  )
            # response = openai.ChatCompletion.create(
            #           model="gpt-3.5-turbo",
            #           messages=message_1,
            #           temperature=1.2,
            #           max_tokens=512,
            #           top_p=1,
            #           n=5,
            #           frequency_penalty=-0.5,
            #           presence_penalty=0.5,
            #     )
            try_nums = 0
            kk_flag = 1
        except Exception as e:
            if 'exceeded your current quota' in str(e):

                # open_ai_keys_index +=1
                openai.api_key = open_ai_keys[open_ai_keys_index]
            time.sleep(1)
            try_nums = try_nums-1
            kk_flag = 0

    if kk_flag == 0:
        time.sleep(5)
        response = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=input_1,
                      max_tokens=256,
                      temperature=0,
                      top_p=1,
                      frequency_penalty=0,
                      presence_penalty=0,
                      n = 1,
                  )
        # response = openai.ChatCompletion.create(
        #               model="gpt-3.5-turbo",
        #               messages=message_1,
        #               temperature=1.2,
        #               max_tokens=512,
        #               top_p=1,
        #               n=5,
        #               frequency_penalty=-0.5,
        #               presence_penalty=0.5,
        #         )

    predictions_1 = response["choices"][0]['text']
    # predictions_1 = response.choices[0].message.content


    input_2 = prompt_2.format(', '.join(candidate_items), ', '.join(seq_list[-length_limit:]), predictions_1)
    message_2 = [
          {"role": "system", "content": "You are a helpful movie recommendation expert."},
          {"role": "user", "content": input_2},
    ]

    try_nums = 5
    kk_flag = 1
    while try_nums:
        try:
            response = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=input_2,
                      max_tokens=512,
                      temperature=0,
                      top_p=1,
                      frequency_penalty=0,
                      presence_penalty=0,
                      n = 1,
                  )
            # response = openai.ChatCompletion.create(
            #           model="gpt-3.5-turbo",
            #           messages=message_2,
            #           temperature=1.2,
            #           max_tokens=512,
            #           top_p=1,
            #           n=5,
            #           frequency_penalty=-0.5,
            #           presence_penalty=0.5,
            #     )
            try_nums = 0
            kk_flag = 1
        except Exception as e:
            if 'exceeded your current quota' in str(e):

                # open_ai_keys_index +=1
                openai.api_key = open_ai_keys[open_ai_keys_index]
            time.sleep(1)
            try_nums = try_nums-1
            kk_flag = 0

    if kk_flag == 0:
        time.sleep(5)
        response = openai.Completion.create(
                      model="text-davinci-003",
                      prompt=input_2,
                      max_tokens=512,
                      temperature=0,
                      top_p=1,
                      frequency_penalty=0,
                      presence_penalty=0,
                      n = 1,
                  )
        # response = openai.ChatCompletion.create(
        #               model="gpt-3.5-turbo",
        #               messages=message_2,
        #               temperature=1.2,
        #               max_tokens=512,
        #               top_p=1,
        #               n=5,
        #               frequency_penalty=-0.5,
        #               presence_penalty=0.5,
        #         )

    predictions = response["choices"][0]['text']
    # predictions = response.choices[0].message.content


    hit_=0
    if elem[1] in predictions:
        count += 1
        hit_ = 1
    else:
        pass
    total +=1



    # print (f"input_1:{input_1}")
    # print (f"predictions_1:{predictions_1}\n")
    # print (f"input_2:{input_2}")
    # print (f"predictions_2:{predictions_2}\n")
    # print (f"input_3:{input_3}")
    print (f"GT:{elem[1]}")
    print (f"predictions:{predictions}")

    # print (f"GT:{elem[-1]}")
    print (f'PID:{i}; count/total:{count}/{total}={count*1.0/total}\n')
    result_json = {"PID": i,
                   "Input_1": input_1,
                   "Input_2": input_2,
                   "GT": elem[1],
                   "Predictions_1": predictions_1,
                   "Predictions": predictions,
                   'Hit': hit_,
                   'Count': count,
                   'Current_total':total,
                   'Hit@10':count*1.0/total}
    results_data.append(result_json)


file_dir = f"/content/drive/MyDrive/Final_Project/result/results_multi_prompting_len{length_limit}_numcand_{total_i}_seed{rseed}.json"
write_json(results_data, file_dir)

GT:Under Siege 2: Dark Territory
predictions:1. Happy Gilmore - Money Train (starring Adam Sandler).
2. Boomerang - The Shadow (starring Eddie Murphy).
3. Made in America - Die Hard 2 (starring Whoopi Goldberg and Ted Danson).
4. Grease 2 - First Knight (starring Michelle Pfeiffer).
5. Michael - Terminal Velocity (starring John Travolta).
6. City Slickers II: The Legend of Curly's Gold - Young Guns (starring Billy Crystal and Daniel Stern).
7. Beverly Hills Cop III - Under Siege 2: Dark Territory (starring Eddie Murphy).
8. Grumpier Old Men - Batman Forever (starring Jack Lemmon and Walter Matthau).
PID:16; count/total:1/1=1.0

GT:Speed
predictions:1. The Maltese Falcon - Star Trek VI: The Undiscovered Country 
2. Con Air - True Lies 
3. Romy and Michele's High School Reunion - The Three Musketeers 
4. Anastasia - Stargate 
5. Grosse Pointe Blank - Money Train 
6. The Fifth Element - GoldenEye 
7. Starship Troopers - Full Metal Jacket 
8. Wild America - Under Siege 
9. Batman Returns -